In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import History
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt


(X_train, y_train), (X_test, y_test) = mnist.Age_Dataset()


num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')


num_classes = 10
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)


def create_model(num_neurons=128, activation='relu', optimizer='adam', lr=0.001):
    model = Sequential()
    model.add(Dense(num_neurons, input_dim=num_pixels, activation=activation))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer(lr=lr), metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=create_model, verbose=0)


param_grid = {
    'num_neurons': [64, 128, 256],
    'activation': ['relu', 'tanh', 'sigmoid'],
    'optimizer': ['adam', 'sgd'],
    'lr': [0.001, 0.01, 0.1],
    'batch_size': [32, 64],
    'epochs': [5, 10]
}


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)


history = History()
grid_result = grid.fit(X_train, y_train, callbacks=[history])


print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


best_model = create_model(num_neurons=grid_result.best_params_['num_neurons'],
                          activation=grid_result.best_params_['activation'],
                          optimizer=grid_result.best_params_['optimizer'],
                          lr=grid_result.best_params_['lr'])


history = best_model.fit(X_train, y_train,
                          batch_size=grid_result.best_params_['batch_size'],
                          epochs=grid_result.best_params_['epochs'],
                          verbose=1,
                          validation_data=(X_test, y_test))


plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()